<a href="https://colab.research.google.com/github/csaybar/EarthEngineMasterGIS/blob/master/EXTRA/extra_joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine-python/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

## **MASTERGIS: Joins**

Los join son una de las herramientas mas potentes de GEE pero tambien una de las mas dificiles de perfeccionar. En esta lectura, aprenderemos sobre:

- Como realizar un simple join.
- Como realizar un Inverted join.
- Como realizar un Save-All join.
- Como realizar un Save-Best join.
- Como realizar un spatial join.

<img src="https://donnierock.files.wordpress.com/2014/03/udqpd.jpg" align="right" width = 60%/>

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"Ingrese_su_TOKEN_AQUI"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
from pprint import pprint
ee.Initialize()

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

Las `joins` se utilizan para combinar elementos de diferentes colecciones (por ejemplo, ImageCollection o FeatureCollection) en base a una condicion especificada por un `ee.Filter`. El filtro se construye con argumentos para las propiedades de cada coleccion que estan relacionadas entre si. En concreto, `leftField` especifica la propiedad de la colección primaria que esta relacionada con la `rightField` de la coleccion secundaria. El tipo de filtro (por ejemplo, `equals`, `greaterThanOrEquals`, `lessThan`, etc.) indica la relacion entre los campos. El tipo de union indica las relaciones de uno a muchos o de uno a uno entre los elementos de las colecciones y cuantas coincidencias hay que retener. El resultado de una union se produce por `join.apply()` y variara segun el tipo de union.

### **1. Simple Join**

Un `simple join` devuelve los elementos de la coleccion primaria que coinciden con cualquier elemento de la coleccion secundaria segun la condicion de coincidencia del filtro. Para realizar un `simple join`, use un  **ee.Join.simple()**. Esto puede ser util para encontrar los elementos comunes entre las diferentes colecciones o para filtrar una coleccion por otra. Por ejemplo, considere dos colecciones de imagenes que (podrian) tener algunos elementos coincidentes, donde la **"coincidencia"** se define por la condicion especificada en un filtro. Por ejemplo, dejemos que la coincidencia signifique que los ID de la imagen son iguales. Dado que las imagenes coincidentes en ambas colecciones son las mismas, utilice un `simple join` para descubrir este conjunto de imagenes coincidentes:

In [0]:
# Cargue una coleccion de imagenes Landsat 8 en un punto de interes.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
               .filterBounds(ee.Geometry.Point(-122.09, 37.42));

# Definir fechas de inicio y finalizacion con las que filtrar las colecciones.
april = '2014-04-01'
may = '2014-05-01'
june = '2014-06-01'
july = '2014-07-01'

# La coleccion principal es imagenes Landsat de abril a junio.
primary = collection.filterDate(april, june)

# La coleccion secundaria es imagenes Landsat de mayo a julio.
secondary = collection.filterDate(may, july)

# Use un filtro igual para definir como coinciden las colecciones.
my_filter = ee.Filter.equals(
    leftField = 'system:index',
    rightField = 'system:index'
)

# Crea el join.
simpleJoin = ee.Join.simple()

# Aplicar el join.
simpleJoined = simpleJoin.apply(primary, secondary, my_filter)

# Mostrar el resultado.
print('Tipo de dato:',type(simpleJoined))
print('Numero de elementos:', simpleJoined.size().getInfo())
print('ID primer imagen:', ee.Feature(simpleJoined.toList(1).get(0)).get("system:index").getInfo())
print('ID segunda imagen:', ee.Feature(simpleJoined.toList(2).get(1)).get("system:index").getInfo())

Esta salida muestra que dos imagenes coinciden (como se especifica en el filtro) entre las colecciones primarias y secundarias, imagenes al dia del año 125 y 141, o 5 y 21 de mayo. Una caracteristica importante a recordar es que siempre el resultado (output) luego de realizar un Join sera un **FeatureCollection** a pesar que se esta trabajando con Imagenes, puede forzar la conversión a un ee.ImageCollection utilizando **`ee.ImageCollection(simpleJoined)`**

In [0]:
#ee.ImageCollection(simpleJoined).first().getInfo()
simpleJoined.first().getInfo()

### **2. Inverted Join**

Suponga que el proposito de la union es retener todas las imagenes en la coleccion primaria que no estan en la coleccion secundaria. Puede realizar este tipo de union invertida utilizando **ee.Join.inverted()**. Usando el filtro, colecciones primarias y secundarias como se define en el ejemplo de combinacion simple, especifique el `inverted join` de la siguiente manera:

In [0]:
# Crear el  join.
invertedJoin = ee.Join.inverted()

# Aplicar el  join
invertedJoined = invertedJoin.apply(primary, secondary, my_filter)

#Mostrar el resultado
print('Tipo de dato:',type(invertedJoined))
print('Numero de elementos:', invertedJoined.size().getInfo())
print('ID primer imagen:', ee.Feature(invertedJoined.toList(1).get(0)).get("system:index").getInfo())
print('ID segunda imagen:', ee.Feature(invertedJoined.toList(2).get(1)).get("system:index").getInfo())

Tipo de dato: <class 'ee.featurecollection.FeatureCollection'>
Numero de elementos: 2
ID primer imagen: LC08_044034_20140403
ID segunda imagen: LC08_044034_20140419


El `inversed join` a diferencia del `simple join` devolvera las imagenes del 3 de abril y del 19 de abril, indicando las imagenes que estan presentes en la coleccion primaria pero no en la secundaria.

### **3. Inner Joins**

Para enumerar todas las coincidencias entre los elementos de dos colecciones, use un `ee.Join.inner()`. La salida de un _inner join_ es un `FeatureCollection` (incluso si se une una `ImageCollection` a otra `ImageCollection`). Cada feature del resultado (output) representa una coincidencia, donde los elementos coincidentes **se almacenan en las propiedades del feature**. Por ejemplo, `feature.get('primary')` es el elemento de la coleccion primaria que coincide con el elemento de la coleccion secundaria almacenado en `feature.get('secundary')`. (Se pueden especificar diferentes nombres para estas propiedades como argumentos de `inner()`, pero `'primary'` y `'secondary'` son los predeterminados). Las relaciones de uno a muchos están representadas por multiples features en la salida. Si un elemento de cualquiera de las dos colecciones no tiene una coincidencia, no estara presente en la salida.

Los ejemplos sobre `join` que utilizan como entrada un `ImageCollection` se aplican sin modificacion a las entradas de `FeatureCollection`. Tambien es posible unir un `FeatureCollection` a un `ImageCollection` y viceversa. Considere el siguiente ejemplo sobre `inner join`:


In [0]:
# Crear una coleccion primaria
primaryFeatures = ee.FeatureCollection([
  ee.Feature(None, {'foo': 0, 'label': 'a'}),
  ee.Feature(None, {'foo': 1, 'label': 'b'}),
  ee.Feature(None, {'foo': 1, 'label': 'c'}),
  ee.Feature(None, {'foo': 2, 'label': 'd'}),
])

# Crear una coleccion secundaria
secondaryFeatures = ee.FeatureCollection([
  ee.Feature(None, {'bar': 1, 'label': 'e'}),
  ee.Feature(None, {'bar': 1, 'label': 'f'}),
  ee.Feature(None, {'bar': 2, 'label': 'g'}),
  ee.Feature(None, {'bar': 3, 'label': 'h'}),
])

# Use un filtro igual para especificar cómo coinciden las colecciones.
toyFilter = ee.Filter.equals(
    leftField = 'foo',
    rightField = 'bar'
)

# Defina un join.
innerJoin = ee.Join.inner('primary', 'secondary')

# Aplique un join.
toyJoin = innerJoin.apply(primaryFeatures, secondaryFeatures, toyFilter)

# Imprima los resultados
toyJoin.getInfo()

En el ejemplo anterior, observe que la relacion entre las tablas esta definida en el filtro, lo que indica que los campos `"foo"` y `"bar"` son los campos de union. A continuacion se especifica un `inner join` y se aplica a las colecciones. Inspeccione la salida y observe que cada posible union se representa como un `feature`.

Para un ejemplo mas interesante, considere "unir" los objetos de MODIS (ImageCollection). Los datos de calidad de MODIS se almacenan a veces en una coleccion separada de los datos de imagen, por lo que un `inner join` es conveniente para unir las dos colecciones a fin de aplicar los datos de calidad. En este caso, los tiempos de adquisicion de la imagen son identicos, por lo que un `filter equals` se encargara de especificar esta relacion entre las dos colecciones:


In [0]:
# Haz un filtro de fechas para obtener imagenes en este rango.
dateFilter = ee.Filter.date('2014-01-01', '2014-02-01')

# Carga los datos EVI de MODIS
mcd43a4 = ee.ImageCollection('MODIS/MCD43A4_006_EVI')\
            .filter(dateFilter)

# Cargar una colección de MODIS con datos de calidad.
mcd43a2 = ee.ImageCollection('MODIS/006/MCD43A2')\
            .filter(dateFilter)

# Defina un Inner Join
innerJoin = ee.Join.inner()

# Especificar un filtro de igualdad para las marcas de tiempo de la imagen.
filterTimeEq = ee.Filter.equals(
  leftField = 'system:time_start',
  rightField = 'system:time_start'
)

# Aplicar el join.
innerJoinedMODIS = innerJoin.apply(mcd43a4, mcd43a2, filterTimeEq)

# Muestre los resultados del join: recuerde que siempre sera un FeatureCollection!
innerJoinedMODIS.getInfo()

Para hacer uso de las imagenes unidas en la salida FeatureCollection, `map()` una funcion de combinacion sobre la salida. Por ejemplo, las imágenes unidas pueden ser apiladas de tal manera que las bandas de calidad se añadan a los datos de la imagen:

In [0]:
# Mapea una funcion para fusionar los resultados en la salida FeatureCollection.
joinedMODIS = innerJoinedMODIS.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary')))

# Imprime el resultado de la fusion.
joinedMODIS.first().getInfo()

El resultado aqui es una ImageCollection a pesar de aparecer como FeatureCollection. Cada imagen en la ImageCollection resultante tiene todas las bandas de las imagenes de la colección primaria (en este ejemplo sólo 'EVI') y todas las bandas de la imagen coincidente de la coleccion secundaria (las bandas de calidad).

### **4. Save-All Joins**

Save-joins es la forma mas comun de representar las relaciones `one-to-many` en Earth Engine. A diferencia de un `inner join`, un
save-join guarda las coincidencias de la colección secundaria como una propiedad nombrada de los `features` de la coleccion primaria. Para guardar todas esas coincidencias, use un `ee.Join.saveAll()`. Si hay una relacion `one-to-many`, un join `saveAll()` almacena todas los features coincidentes como una `ee.List`. Los elementos no coincidentes de la coleccion primaria se eliminan. Por ejemplo, supongamos que es necesario obtener todas las imágenes MODIS adquiridas en un plazo de dos dias a partir de cada imagen Landsat de una coleccion. Este ejemplo utiliza un join `saveAll()` para ese propósito:


In [0]:
# Cargar la collecion primaria: Landsat .
primary = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
            .filterDate('2014-04-01', '2014-06-01')\
            .filterBounds(ee.Geometry.Point(-122.092, 37.42))

# Cargar la collecion secundaria: MODIS.
modSecondary = ee.ImageCollection('MODIS/006/MOD09GA')\
                 .filterDate('2014-03-01', '2014-07-01')

# Defina una diferencia de tiempo permitida: dos días en milisegundos.
twoDaysMillis = 2 * 24 * 60 * 60 * 1000

# Cree un filtro de tiempo para definir una coincidencia como marcas de tiempo superpuestas.
timeFilter = ee.Filter.Or(
  ee.Filter.maxDifference(
    difference = twoDaysMillis,
    leftField = 'system:time_start',
    rightField = 'system:time_end'
  ),
  ee.Filter.maxDifference(
    difference = twoDaysMillis,
    leftField = 'system:time_end',
    rightField = 'system:time_start'
 )
)

# Defina el join
saveAllJoin = ee.Join.saveAll(
  matchesKey =  'terra',
  ordering = 'system:time_start',
  ascending =  True
)

# Aplique el join
landsatModis = saveAllJoin.apply(primary, modSecondary, timeFilter)

# Display los resultados.
ee.Image(landsatModis.first()).getInfo()

En este ejemplo, notese que la **coleccion secundaria** de MODIS esta prefiltrada para ser cronologicamente similar a la coleccipn primaria de Landsat para mayor eficiencia. Para comparar el tiempo de adquisicion del Landsat con el tiempo compuesto del MODIS, que tiene un rango diario el filtro compara los puntos finales de las marcas de tiempo de la imagen. El join se define con el nombre de la propiedad utilizada para almacenar la lista de coincidencias para cada imagen Landsat ('terra') y el parametro opcional para ordenar la lista de coincidencias por la propiedad `system:time_start`.

La inspeccion del resultado indica que las imagenes de la coleccion primaria tienen la propiedad de tierra añadida que almacena una lista de las imagenes MODIS coincidentes.



### **5. Save-Best Joins**

Para guardar solo la mejor coincidencia de cada elemento de una coleccion, usa un `ee.Join.saveBest()`. La funcion join `saveBest()` funciona de forma equivalente a `saveAll()`, excepto que para cada elemento de la coleccion primaria, guarda el elemento de la coleccion secundaria con la mejor coincidencia. Los elementos no coincidentes en la coleccion primaria se eliminan. Supongamos que la intencion es encontrar una imagen meteorologica mas cercana en el tiempo a cada imagen de Landsat en la colección primaria. Para realizar esta union, el `ee.Filter` debe ser redefinido para una sola condicion de union (los filtros combinados no funcionaran con saveBest() ya que es ambiguo como combinar rangos de multiples sub-filtros):



In [0]:
# Load a primary collection: Landsat imagery.
primary = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
            .filterDate('2014-04-01', '2014-06-01')\
            .filterBounds(ee.Geometry.Point(-122.092, 37.42))

# Load a secondary collection: GRIDMET meteorological data
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')

# Define a max difference filter to compare timestamps.
maxDiffFilter = ee.Filter.maxDifference(
  difference = 2 * 24 * 60 * 60 * 1000,
  leftField = 'system:time_start',
  rightField = 'system:time_start'
)

# Define the join.
saveBestJoin = ee.Join.saveBest(
  matchKey = 'bestImage',
  measureKey = 'timeDiff'
)

# Apply the join.
landsatMet = saveBestJoin.apply(primary, gridmet, maxDiffFilter)

# Print the result
landsatMet.first().getInfo()

Notese que una union `saveBest()` define el nombre de la propiedad con la que se almacena la mejor coincidencia (`'bestImage'`) y el nombre de la propiedad con la que se almacena la fiabilidad de la metrica de la coincidencia ('timeDiff'). La inspeccion de los resultados indica que se ha añadido una imagen DAYMET coincidente a la propiedad `bestImage` para cada escena Landsat de la colección primaria. Cada una de estas imagenes `DAYMET` tiene la propiedad `timeDiff` que indica la diferencia de tiempo en milisegundos entre la imagen DAYMET y la imagen Landsat, que sera minima entre las imagenes `DAYMET` que pasan la condicion en el filtro.


### **6. Spatial Joins**

Las colecciones pueden unirse por su ubicacion espacial asi como por los valores de las propiedades. Para unirlas en base a la ubicacion espacial utilice un filtro `withinDistance()` con los campos de join `.geo` especificados. El campo `.geo` indica que la geometria del elemento debe utilizarse para calcular la metrica de la distancia. Por ejemplo, considere la tarea de encontrar todas las plantas de energia dentro de los 100 kilometros del Parque Nacional Yosemite, EE.UU. Para ello, utilice un filtro en los campos de geometria, con la distancia maxima establecida en 100 kilometros utilizando el parametro de distancia:


In [0]:
# Cargamos la collecion primaria: areas protegidas (Yosemite National Park).
primary = ee.FeatureCollection("WCMC/WDPA/current/polygons")\
            .filter(ee.Filter.eq('NAME', 'Yosemite National Park'))

# Cargamos la coleccion secundaria: plantas de energia.
powerPlants = ee.FeatureCollection('WRI/GPPD/power_plants')

# Definimos un filter espacial, con distancia de 100 km.
distFilter = ee.Filter.withinDistance(
  distance = 100000,
  leftField = '.geo',
  rightField = '.geo',
  maxError = 10
)

# Defina un saveAll join.
distSaveAll = ee.Join.saveAll(
  matchesKey = 'points',
  measureKey = 'distance'
)

# Aplica los join.
spatialJoined = distSaveAll.apply(primary, powerPlants, distFilter)

# Imprima los resultados.
spatialJoined.first().getInfo()

Observe que el ejemplo anterior une un `FeatureCollection` con otro `FeatureCollection`. El join `saveAll()` establece una propiedad (`point`) en cada feature de la coleccion `primaria` que almacena una lista de los puntos en un radio de 100 km del `feature`. La distancia de cada punto al `feature` se almacena en la propiedad de distancia de cada punto unido.

Las uniones espaciales tambien pueden utilizarse para identificar que rasgos de una coleccion se cruzan con los de otra. Por ejemplo, consideremos dos colecciones de rasgos: una `coleccion primaria` que contiene poligonos que representan los limites de los estados de los `EE.UU`, y una `coleccion secundaria` que contiene ubicaciones de puntos que representan centrales electricas. Supongamos que es necesario determinar el numero que intersecta cada estado. Esto se puede lograr con una union espacial como la siguiente:

In [0]:
# Cargue la colección primaria: limites de estados en EEUU.
states = ee.FeatureCollection('TIGER/2018/States')

# Cargue la colección secundaria: plantas de energia.
powerPlants = ee.FeatureCollection('WRI/GPPD/power_plants')

# Defina un filtro espacial como geometrías que se cruzan.
spatialFilter = ee.Filter.intersects(
  leftField = '.geo',
  rightField = '.geo',
  maxError = 10
)

# Defina un save all join.
saveAllJoin = ee.Join.saveAll(
  matchesKey = 'power_plants',
)

# Aplique el join.
intersectJoined = saveAllJoin.apply(states, powerPlants, spatialFilter)

# Agregue el conteo de plantas de energia por estado como propiedad.
def npowerplants(state):
  # Obtenga la lista de intersecciones "power_plant", cuente cuántas intersecaron este estado.
  nPowerPlants = ee.List(state.get('power_plants')).size()
  # Devuelve la función de estado con una nueva propiedad: recuento de plantas de energía.
  return state.set('n_power_plants', nPowerPlants)

intersectJoined = intersectJoined.map(npowerplants)